In [1]:
from datetime import datetime
import os
import pandas
import re

In [2]:
EX_RUNTIME = 300 # seconds

In [15]:
dir = os.scandir('raw/')
experiments = list(filter(lambda x: x.is_dir() and x.name[0] != '.', dir))
print(experiments)

[<DirEntry 'oayotman'>, <DirEntry 'ghtacefn'>, <DirEntry 'alqxmewb'>, <DirEntry 'alqxmewb2'>, <DirEntry 'alqxmewb-batch1'>]


In [22]:
def experiment_to_csv(path):
    jobs = pandas.read_csv(f'{experiment.path}/design20epochs.csv')
    jobs.columns.values[0] = 'Index'
    jobs = jobs[jobs.Index >= 0]
    
    jobs['accuracy'] = 0.0
    jobs['training_time'] = 0.0
    #jobs['epochs'] = 0
    pat = re.compile("accuracy: (\d+\.\d+)")
    pat2 = re.compile("Wall clock time is (\d+\.\d+)")
    for idx, row in jobs.iterrows():
    #     {int(row.num_nodes)}
        row.failure_rate = 0.0
        row.failure_duration = 0.0
        filename = f"{int(row.Index)}-nodes8-batch{row.batch_size}-epochs{row.max_epochs}-lrate{row.learning_rate}-memory{row.memory_size}-{row.model}.log"


        try:
            with open(f'{experiment.path}/{filename}', 'r') as file:
                content = file.read()
                result = pat.findall(content)
                result2 = pat2.findall(content)
                jobs.at[idx, 'accuracy'] = result[-1] if len(result) > 0 else 0
                jobs.at[idx, 'training_time'] = result2[-1] if len(result) > 0 else 0
                #jobs.loc[idx, 'epochs'] = len(result)
        except FileNotFoundError as e:
            print(e)
            jobs.at[idx, 'accuracy'] = -1
            jobs.at[idx, 'training_time'] = -1
            #jobs.loc[idx, 'epochs'] = -1
    
    time = os.stat(f'{experiment.path}/design20epochs.csv').st_mtime
    dt = datetime.fromtimestamp(time)
    jobs.to_csv(f'ex-{experiment}-{dt.strftime("%Y-%m-%d_%H:%M")}.csv')
    jobs
    print(jobs)

In [27]:
#for experiment in experiments:
experiment = experiments[0]
if os.path.exists(f'{experiment.path}/design20epochs.csv'):
    experiment_to_csv(experiment.path)

    Index  batch_size  max_epochs  learning_rate  memory_size      model  \
0     160         128          20          0.010         6000  lenet5.py   
1     161         128          20          0.010         6000  bi-rnn.py   
2     162         128          20          0.010        12000  lenet5.py   
3     163         128          20          0.010        12000  bi-rnn.py   
4     164         128          20          0.005         6000  lenet5.py   
..    ...         ...         ...            ...          ...        ...   
75    235         256          20          0.010        12000  bi-rnn.py   
76    236         256          20          0.005         6000  lenet5.py   
77    237         256          20          0.005         6000  bi-rnn.py   
78    238         256          20          0.005        12000  lenet5.py   
79    239         256          20          0.005        12000  bi-rnn.py   

    accuracy  training_time  
0     0.9581  530781.844851  
1     0.6701  884054.183743